In [1]:
!pip install sentence-transformers pandas tqdm

In [43]:
!python -m spacy download en_core_web_md
!pip install keybert


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 66.2 MB/s eta 0:00:00m eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [2]:
!pip install "elasticsearch<9"

In [23]:
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
import pandas as pd
from tqdm import tqdm
import re
import spacy

In [4]:
es = Elasticsearch(
    hosts=["http://localhost:9200"],
    # Nếu không cần auth thì không cần user/pass
)
# Test kết nối
#assert es.ping(), "Elasticsearch không kết nối được!"


In [5]:
import requests
print(requests.get("http://localhost:9200").text)


{
  "name" : "941bc3af368d",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "6KQrMvN8QuyudDcEoPEIBA",
  "version" : {
    "number" : "8.13.0",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "09df99393193b2c53d92899662a8b8b3c55b45cd",
    "build_date" : "2024-03-22T03:35:46.757803203Z",
    "build_snapshot" : false,
    "lucene_version" : "9.10.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}



In [6]:
index_name = "places_danang"

mapping = {
    "mappings": {
        "properties": {
            "type": {"type": "keyword"},
            "name": {"type": "text"},
            "description": {"type": "text"},
            "time": {"type": "keyword"},
            "price": {"type": "keyword"},
            "location": {"type": "text"},
            "area": {"type": "keyword"},
            "note": {"type": "text"},
            "id": {"type": "keyword"},
            "full_text": {"type": "text"},  
            "vector_search": {
                "type": "dense_vector",
                "dims": 384,  
                "index": True,           
                "similarity": "cosine"  
            }
        }
    }
}


In [8]:
try:
    es.indices.create(index=index_name, body=mapping)
except Exception as e:
    print("Mapping error details:", getattr(e, 'info', str(e)))

Mapping error details: {'error': {'root_cause': [{'type': 'resource_already_exists_exception', 'reason': 'index [places_danang/SeEZJwL5SbqvNzcifcZgYQ] already exists', 'index_uuid': 'SeEZJwL5SbqvNzcifcZgYQ', 'index': 'places_danang'}], 'type': 'resource_already_exists_exception', 'reason': 'index [places_danang/SeEZJwL5SbqvNzcifcZgYQ] already exists', 'index_uuid': 'SeEZJwL5SbqvNzcifcZgYQ', 'index': 'places_danang'}, 'status': 400}


In [9]:
#Delete pervious index and create a new one: 
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)
es.indices.create(index=index_name, body=mapping)
print(f"Index `{index_name}` đã được tạo!")


Index `places_danang` đã được tạo!


In [10]:
model = SentenceTransformer('all-MiniLM-L6-v2')


In [11]:
df = pd.read_csv("data_danang_ok.csv")  # Đường dẫn file của bạn

# Tạo embedding cho từng record
def embed(text):
    return model.encode(text).tolist()

# Nếu cột vector_search đã có, bỏ qua đoạn này, còn không:
tqdm.pandas()
df["vector_search"] = df["full_text"].progress_apply(embed)

#indexing data to elasticsearch
for i, row in tqdm(df.iterrows(), total=len(df)):
    doc = row.to_dict()
    # Nếu vector_search dạng numpy, cần chuyển sang list
    es.index(index=index_name, id=doc["id"], document=doc)

100%|████████████████████████████████████████| 299/299 [00:01<00:00, 231.67it/s]


**Querry processing for better search**

In [47]:
'''
from keybert import KeyBERT
kw_model = KeyBERT(model='all-MiniLM-L6-v2')  # Dùng chính model bạn dùng cho embedding

def extract_keyphrases(query, top_n=3):
    keywords = kw_model.extract_keywords(
        query, 
        top_n=top_n, 
        stop_words="english", 
        use_maxsum=True,    # Thử các phương pháp diversity
        nr_candidates=20,   # Số candidate nhiều hơn
        ngram_range=(2, 4)  # Xét cả bigram, trigram, fourgram
    )
    return [k[0] for k in keywords]

print(extract_keyphrases("Where to stay near My Khe Beach"))
# Nhiều khả năng sẽ trả về ['my khe beach', ...]
'''


'\nfrom keybert import KeyBERT\nkw_model = KeyBERT(model=\'all-MiniLM-L6-v2\')  # Dùng chính model bạn dùng cho embedding\n\ndef extract_keyphrases(query, top_n=3):\n    keywords = kw_model.extract_keywords(\n        query, \n        top_n=top_n, \n        stop_words="english", \n        use_maxsum=True,    # Thử các phương pháp diversity\n        nr_candidates=20,   # Số candidate nhiều hơn\n        ngram_range=(2, 4)  # Xét cả bigram, trigram, fourgram\n    )\n    return [k[0] for k in keywords]\n\nprint(extract_keyphrases("Where to stay near My Khe Beach"))\n# Nhiều khả năng sẽ trả về [\'my khe beach\', ...]\n'

In [34]:
STOP_WORDS = {
    "a", "an", "and", "are", "as", "at", "be", "but", "by", "for", "if",
    "in", "into", "is", "it", "no", "not", "of", "on", "or", "such", "that",
    "the", "their", "then", "there", "these", "they", "this", "to", "was",
    "will", "with", "me", "my", "you", "your", "we", "our", "us", "he",
    "him", "his", "she", "her", "hers", "it", "its", "them", "so", "too"
}

def preprocess_query_for_vector(query):
    # Bỏ dấu câu (tuỳ chọn, để nguyên cũng được vì embedding model hiểu)
    query_no_punct = re.sub(r'[^\w\s]', '', query)
    # Bỏ stopword, giữ lại trật tự và ý nghĩa câu
    words = query_no_punct.split()
    filtered = [w for w in words if w.lower() not in STOP_WORDS]
    # Ghép lại thành câu ngắn gọn
    processed_query = " ".join(filtered) if filtered else query
    return processed_query

# Ví dụ:
# "Where to eat Bun Bo Hue in the evening?" -> "Where eat Bun Bo Hue evening"

**Search**

In [48]:
def bm25_search(query, top_k=10):
    body = {
        "size": top_k,
        "query": {
            "multi_match": {
                "query": query,
                "fields": ["full_text"],
                #"operator": "or",
            }
        }
    }
    res = es.search(index=index_name, body=body)
    return [
        {
            "id": hit["_source"]["id"],
            "score": hit["_score"],
            "full_text": hit["_source"]["full_text"]
        }
        for hit in res["hits"]["hits"]
    ]


In [55]:
def vector_search(query, top_k=10):
    query_vec = model.encode(query).tolist()
    body = {
        "size": top_k,
        "query": {
            "script_score": {
                "query": {"match_all": {}},
                "script": {
                    "source": "cosineSimilarity(params.query_vector, 'vector_search') + 1.0",
                    "params": {"query_vector": query_vec}
                }
            }
        }
    }
    res = es.search(index=index_name, body=body)
    return [
        {
            "id": hit["_source"]["id"],
            "score": hit["_score"],
            "full_text": hit["_source"]["full_text"]
        }
        for hit in res["hits"]["hits"]
    ]


In [61]:
def reciprocal_rank_fusion(lexical_hits, semantic_hits, k=60, top_k=5):
    rrf_scores = {}
    # Process lexical hits
    for rank, hit in enumerate(lexical_hits, start=1):
        doc_id = hit["id"]
        score = 1 / (k + rank)
        if doc_id in rrf_scores:
            rrf_scores[doc_id]["rrf_score"] += score
            rrf_scores[doc_id]["lexical_score"] = hit["score"]
        else:
            rrf_scores[doc_id] = {
                "full_text": hit.get("full_text", ""),
                "lexical_score": hit["score"],
                "semantic_score": 0,
                "rrf_score": score
            }
    # Process semantic hits
    for rank, hit in enumerate(semantic_hits, start=1):
        doc_id = hit["id"]
        score = 1 / (k + rank)
        if doc_id in rrf_scores:
            rrf_scores[doc_id]["rrf_score"] += score
            rrf_scores[doc_id]["semantic_score"] = hit["score"]
        else:
            rrf_scores[doc_id] = {
                "full_text": hit.get("full_text", ""),
                "lexical_score": 0,
                "semantic_score": hit["score"],
                "rrf_score": score
            }
    # Sort by rrf_score
    results = sorted(rrf_scores.values(), key=lambda x: x["rrf_score"], reverse=True)[:top_k]
    return results


In [62]:
'''
def hybrid_search(query, top_k=5, alpha=0.5):
    # BM25
    bm25_results = bm25_search(query, top_k=top_k*2)
    bm25_ids = {doc["id"]: doc for doc in bm25_results}

    # Vector
    vec_results = vector_search(query, top_k=top_k*2)
    vec_ids = {doc["id"]: doc for doc in vec_results}

    # Gộp tất cả id
    all_ids = set(bm25_ids.keys()) | set(vec_ids.keys())

    # Tính điểm hybrid
    hybrid_results = []
    for id_ in all_ids:
        bm25_score = bm25_ids.get(id_, {}).get("score", 0)
        vec_score = vec_ids.get(id_, {}).get("score", 0)
        score = (1 - alpha) * bm25_score + alpha * vec_score
        hybrid_results.append({
            "id": id_,
            "hybrid_score": score,
            "full_text": bm25_ids.get(id_, vec_ids.get(id_, {})).get("full_text", "")
        })

    # Sort theo điểm hybrid
    hybrid_results = sorted(hybrid_results, key=lambda x: x["hybrid_score"], reverse=True)[:top_k]
    return hybrid_results
'''
def hybrid_search(query, top_k=5, k_rrf=60):
    bm25_results = bm25_search(query, top_k=top_k*2)   # Lấy nhiều hơn để RRF hiệu quả hơn
    vector_results = vector_search(query, top_k=top_k*2)
    results = reciprocal_rank_fusion(bm25_results, vector_results, k=k_rrf, top_k=top_k)
    return results


In [63]:
query = "Where to eat Bun Bo Hue in the evening"
#print("BM25:", bm25_search(query)[1]['full_text'])
#print("Vector:", vector_search(query)[0]['full_text'])
print("Hybrid:", hybrid_search(query)[0]['full_text'])


Hybrid: hue-style spicy beef noodle soup morning 35k VND (~1.4$) 23 Ngo Gia Tu center Bun Bo Ba Thuong offers spicy broth with lemongrass, beef shank, and thick round noodles.


In [31]:
nlp = spacy.load("en_core_web_md")

def preprocess_query_for_bm25(query, min_word_len=2):
    # Làm sạch: bỏ dấu câu, lowercase
    cleaned = re.sub(r'[^\w\s]', '', query).lower()
    # Loại stopword, từ ngắn
    words = cleaned.split()
    filtered_words = [word for word in words if word not in STOP_WORDS and len(word) >= min_word_len]
    cleaned_query = " ".join(filtered_words) if filtered_words else cleaned
    # Trích xuất noun phrase từ cleaned_query
    doc = nlp(cleaned_query)
    noun_phrases = [chunk.text.strip() for chunk in doc.noun_chunks if chunk.text.strip()]
    # Nếu có noun phrase thì trả về list đó, còn không thì trả về cleaned_query
    return noun_phrases if noun_phrases else [cleaned_query]

# Ví dụ:
# "Where to eat Bun Bo Hue in the evening?" -> ['bun bo hue', 'evening']
